In [173]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [174]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
from scipy import stats # type: ignore
import torch # type: ignore

from sklearn.decomposition import PCA # type: ignore
from sklearn.cross_decomposition import PLSRegression # type: ignore


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [175]:
train_pro  =  pd.read_csv("/Users/harsharajkumar/Downloads/project /train_proteins.csv")
train_pep =  pd.read_csv("/Users/harsharajkumar/Downloads/project /train_peptides.csv")
train_cli = pd.read_csv("/Users/harsharajkumar/Downloads/project /train_clinical_data.csv")
supclinical = pd.read_csv("/Users/harsharajkumar/Downloads/project /supplemental_clinical_data.csv")

In [176]:
supclinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


# **EDA OF PROTIENS**

In [177]:
train_pro.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [178]:
train_pro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232741 entries, 0 to 232740
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   visit_id     232741 non-null  object 
 1   visit_month  232741 non-null  int64  
 2   patient_id   232741 non-null  int64  
 3   UniProt      232741 non-null  object 
 4   NPX          232741 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.9+ MB


In [179]:
train_pro.describe()

,visit_month,patient_id,NPX
count,232741.000000,232741.000000,2.327410e+05
mean,26.099205,32593.881873,2.712077e+06
std,22.874719,18608.479506,2.224155e+07
min,0.000000,55.000000,8.460820e+01
25%,6.000000,16566.000000,2.946440e+04
50%,24.000000,29313.000000,1.135560e+05
75%,48.000000,49995.000000,5.638940e+05
max,108.000000,65043.000000,6.138510e+08


In [180]:
train_pro.shape

(232741, 5)

In [181]:
train_pro.isnull().sum()

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64

In [182]:
#sns.pairplot(train_pro)

# **EDA OF PEPTIDES**

In [183]:
train_pep.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [184]:
train_pep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981834 entries, 0 to 981833
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   visit_id          981834 non-null  object 
 1   visit_month       981834 non-null  int64  
 2   patient_id        981834 non-null  int64  
 3   UniProt           981834 non-null  object 
 4   Peptide           981834 non-null  object 
 5   PeptideAbundance  981834 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 44.9+ MB


In [185]:
train_pep.shape

(981834, 6)

In [186]:
train_pep.isnull().sum()

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64

In [187]:
train_pep.describe()

,visit_month,patient_id,PeptideAbundance
count,981834.000000,981834.000000,9.818340e+05
mean,26.105061,32603.465361,6.428902e+05
std,22.913897,18605.934422,3.377989e+06
min,0.000000,55.000000,1.099850e+01
25%,6.000000,16566.000000,2.817425e+04
50%,24.000000,29313.000000,7.430830e+04
75%,48.000000,49995.000000,2.213388e+05
max,108.000000,65043.000000,1.787520e+08


In [188]:
#sns.pairplot(train_pep)

# **EDA OF CLINICAL**

In [189]:
train_cli.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [190]:
train_cli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2615 non-null   object 
 1   patient_id                           2615 non-null   int64  
 2   visit_month                          2615 non-null   int64  
 3   updrs_1                              2614 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2590 non-null   float64
 6   updrs_4                              1577 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 163.6+ KB


In [191]:
train_cli.shape

(2615, 8)

In [192]:
train_cli.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [193]:
train_cli.describe()

,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
count,2615.000000,2615.000000,2614.000000,2613.00000,2590.000000,1577.000000
mean,32651.743786,31.190822,7.110559,6.74359,19.421236,1.861763
std,18535.758700,25.199053,5.525955,6.32323,15.000289,3.022112
min,55.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,16574.000000,10.500000,3.000000,1.00000,6.000000,0.000000
50%,29417.000000,24.000000,6.000000,5.00000,19.000000,0.000000
75%,50611.000000,48.000000,10.000000,10.00000,29.000000,3.000000
max,65043.000000,108.000000,33.000000,40.00000,86.000000,20.000000


In [194]:
#sns.pairplot(train_cli)

While updrs_1 is a reasonable choice for a target variable if it reflects disease progression, consider the factors mentioned above to ensure it aligns with your research question and data. You might need to explore and potentially transform other available variables in the dataset to create a more informative target variable.

In [195]:
"""# Function to merge clinical data, including data cleaning
def merge_clinical(tmp_pep, tmp_pro, tmp_cli):
    # Combine data
    result = pd.merge(tmp_pep, tmp_pro, on=['visit_id', 'visit_month', 'patient_id', 'UniProt'])
    result = pd.merge(result, tmp_cli, on=['visit_id', 'visit_month', 'patient_id'])

    # Convert categorical variables to numerical values
    le = LabelEncoder()
    le.fit(result['UniProt'])
    result['UniProt'] = le.transform(result['UniProt'])
    le.fit(result['Peptide'])
    result['Peptide'] = le.transform(result['Peptide'])

    # **Data Cleaning: Handle potential string values like "Off"**
    # Assuming "Off" might be in a column named "Status" (replace with the actual column name)
    filtered_cols = ['Status']  # List the columns to check for "Off" (adjust as needed)
    for col in filtered_cols:
        if result[col].dtype == object:  # Check if the data type is object (potentially containing strings)
            # Option 1: Remove rows with "Off" (if irrelevant or errors)
            # result = result[result[col] != 'Off']

            # Option 2: Replace "Off" with a more appropriate value (e.g., mean)
            from sklearn.impute import SimpleImputer
            imputer = SimpleImputer(strategy='mean')
            result[col] = imputer.fit_transform(result[[col]])[:, 0]  # Impute only the specific column

    return result

# Merge data with cleaning
mg_table = merge_clinical(train_pep, train_pro, train_cli)"""

'# Function to merge clinical data, including data cleaning\ndef merge_clinical(tmp_pep, tmp_pro, tmp_cli):\n    # Combine data\n    result = pd.merge(tmp_pep, tmp_pro, on=[\'visit_id\', \'visit_month\', \'patient_id\', \'UniProt\'])\n    result = pd.merge(result, tmp_cli, on=[\'visit_id\', \'visit_month\', \'patient_id\'])\n\n    # Convert categorical variables to numerical values\n    le = LabelEncoder()\n    le.fit(result[\'UniProt\'])\n    result[\'UniProt\'] = le.transform(result[\'UniProt\'])\n    le.fit(result[\'Peptide\'])\n    result[\'Peptide\'] = le.transform(result[\'Peptide\'])\n\n    # **Data Cleaning: Handle potential string values like "Off"**\n    # Assuming "Off" might be in a column named "Status" (replace with the actual column name)\n    filtered_cols = [\'Status\']  # List the columns to check for "Off" (adjust as needed)\n    for col in filtered_cols:\n        if result[col].dtype == object:  # Check if the data type is object (potentially containing strings)\n  

In [196]:
# Model creation
def merge_clinical(tmp_pep, tmp_pro, tmp_cli):
    # Combine data
    result = pd.merge(tmp_pep, tmp_pro, on=['visit_id', 'visit_month', 'patient_id', 'UniProt'])
    result = pd.merge(result, tmp_cli, on=['visit_id', 'visit_month', 'patient_id'])
    # Convert categorical variables to numerical values ​​(this time performed for UniProt and Peptide)
    le = LabelEncoder()
    le = le.fit(result['UniProt'])
    result['UniProt'] = le.transform(result['UniProt'])
    le = le.fit(result['Peptide'])
    result['Peptide'] = le.transform(result['Peptide'])
    #le = le.fit(result['upd23b_clinical_state_on_medication'])
    #result['upd23b_clinical_state_on_medication'] = le.transform(result['upd23b_clinical_state_on_medication'])

    return result

mg_table = merge_clinical(train_pep, train_pro, train_cli)




In [197]:
mg_table = mg_table.drop("upd23b_clinical_state_on_medication",axis = 1)

In [198]:
mg_table

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,NPX,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,0,55,0,592,11254.30,11254.3,10.0,6.0,15.0,NaN
1,55_0,0,55,1,301,102060.00,732430.0,10.0,6.0,15.0,NaN
2,55_0,0,55,1,364,174185.00,732430.0,10.0,6.0,15.0,NaN
3,55_0,0,55,1,433,27278.90,732430.0,10.0,6.0,15.0,NaN
4,55_0,0,55,1,744,30838.70,732430.0,10.0,6.0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
941739,58648_108,108,58648,222,375,202820.00,369437.0,6.0,0.0,0.0,NaN
941740,58648_108,108,58648,223,723,105830.00,105830.0,6.0,0.0,0.0,NaN
941741,58648_108,108,58648,225,459,21257.60,21257.6,6.0,0.0,0.0,NaN
941742,58648_108,108,58648,226,21,5127.26,17953.1,6.0,0.0,0.0,NaN


In [199]:
mg_table.isna().sum()

visit_id                 0
visit_month              0
patient_id               0
UniProt                  0
Peptide                  0
PeptideAbundance         0
NPX                      0
updrs_1                  0
updrs_2                  0
updrs_3               9120
updrs_4             446214
dtype: int64

In [200]:
mg_table.shape

(941744, 11)

In [201]:
mg_table = mg_table.dropna()

In [202]:
mg_table.shape

(490952, 11)

In [203]:
#sns.pairplot(mg_table)

In [204]:
# sns.boxplot(mg_table )

In [205]:
X = mg_table.drop("updrs_1", axis=1)
y = mg_table["updrs_1"]


In [206]:
X

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,NPX,updrs_2,updrs_3,updrs_4
931,1517_0,0,1517,0,592,11648.9,11648.9,6.0,25.0,5.0
932,1517_0,0,1517,1,301,63593.4,419015.0,6.0,25.0,5.0
933,1517_0,0,1517,1,364,99566.6,419015.0,6.0,25.0,5.0
934,1517_0,0,1517,1,433,16351.0,419015.0,6.0,25.0,5.0
935,1517_0,0,1517,1,744,15566.0,419015.0,6.0,25.0,5.0
...,...,...,...,...,...,...,...,...,...,...
940831,55096_108,108,55096,222,279,33929.8,123192.0,6.0,46.0,0.0
940832,55096_108,108,55096,222,375,68513.3,123192.0,6.0,46.0,0.0
940833,55096_108,108,55096,223,723,101061.0,101061.0,6.0,46.0,0.0
940834,55096_108,108,55096,224,430,18108.9,18108.9,6.0,46.0,0.0


In [207]:
#sns.boxplot(X)

In [208]:
#sns.pairplot(X)

In [209]:
y

931       11.0
932       11.0
933       11.0
934       11.0
935       11.0
          ... 
940831     5.0
940832     5.0
940833     5.0
940834     5.0
940835     5.0
Name: updrs_1, Length: 490952, dtype: float64

In [210]:
"""from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split  # Import for train/test split (optional)

# Identify categorical columns
categorical_cols = [col for col in X.columns if X[col].dtype == object]

# One-Hot Encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Encode categorical features
X_encoded = pd.concat([X.drop(categorical_cols, axis=1),
                        pd.DataFrame(encoder.fit_transform(X[categorical_cols]))], axis=1)"""


"from sklearn.preprocessing import OneHotEncoder\nfrom sklearn.model_selection import train_test_split  # Import for train/test split (optional)\n\n# Identify categorical columns\ncategorical_cols = [col for col in X.columns if X[col].dtype == object]\n\n# One-Hot Encoder\nencoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')\n\n# Encode categorical features\nX_encoded = pd.concat([X.drop(categorical_cols, axis=1),\n                        pd.DataFrame(encoder.fit_transform(X[categorical_cols]))], axis=1)"

In [211]:
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X_encoded)

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [213]:
X_train

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,NPX,updrs_2,updrs_3,updrs_4
728046,16778_48,48,16778,67,340,85752.8,2481880.0,9.0,28.0,0.0
443604,14242_24,24,14242,87,451,17088.3,93943.5,4.0,15.0,1.0
443262,14242_24,24,14242,30,552,9043080.0,13723700.0,4.0,15.0,1.0
274321,13618_12,12,13618,177,81,54132.7,207375.0,19.0,39.0,3.0
771164,40022_48,48,40022,65,407,3118950.0,19794000.0,7.0,32.0,0.0
...,...,...,...,...,...,...,...,...,...,...
595030,16931_36,36,16931,201,168,24701.9,24701.9,14.0,36.0,0.0
754545,27715_48,48,27715,92,481,37068.7,2599400.0,0.0,0.0,0.0
408771,15504_18,18,15504,178,674,121805.0,646663.0,0.0,5.0,0.0
432729,10715_24,24,10715,16,598,10303.4,53315.5,6.0,27.0,0.0


#applying randomn forest model

In [214]:
# Define Random Forest hyperparameters
n_estimators = 100  # Number of trees in the forest
max_depth = 5  # Maximum depth of individual trees

# Create Random Forest Classifier object
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

# Train the model on the prepared data
clf.fit(X_train, y_train)

# Make predictions on the testing set (optional, for evaluation)
y_pred = clf.predict(X_test)

# Evaluate the model (example using accuracy)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.46484912059150024


now lets fine tune  so that we can get a better model instead of changing parameters which can increase the speed instead of using 1000 trees and getting better accuacy because it takes lot of time to iterate 

In [215]:
"""from sklearn.model_selection import GridSearchCV

# Define the parameter grid as a dictionary
param_grid = {
    'n_estimators': [100]  # Wrap the value in a list
      # Example of different depths to try
}

# Create the GridSearchCV object with your Random Forest Classifier
#rf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)

# Now you can fit the model on the training data
#rf_Grid.fit(X_train, y_train)"""

"from sklearn.model_selection import GridSearchCV\n\n# Define the parameter grid as a dictionary\nparam_grid = {\n    'n_estimators': [100]  # Wrap the value in a list\n      # Example of different depths to try\n}\n\n# Create the GridSearchCV object with your Random Forest Classifier\n#rf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)\n\n# Now you can fit the model on the training data\n#rf_Grid.fit(X_train, y_train)"

In [216]:
# Create the GridSearchCV object with your Random Forest Classifier
#rf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)


In [217]:
# Now you can fit the model on the training data
#rf_Grid.fit(X_train, y_train)

In [218]:
"""max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]"""


"max_features = ['auto', 'sqrt']\n# Maximum number of levels in tree\nmax_depth = [2,4]\n# Minimum number of samples required to split a node\nmin_samples_split = [2, 5]\n# Minimum number of samples required at each leaf node\nmin_samples_leaf = [1, 2]\n# Method of selecting samples for training each tree\nbootstrap = [True, False]"

In [219]:
"""param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)"""


"param_grid = {'n_estimators': n_estimators,\n               'max_features': max_features,\n               'max_depth': max_depth,\n               'min_samples_split': min_samples_split,\n               'min_samples_leaf': min_samples_leaf,\n               'bootstrap': bootstrap}\nprint(param_grid)"

In [220]:
#rf_model = RandomForestClassifier()

In [221]:
#from sklearn.model_selection import GridSearchCV
#rf_Grid = GridSearchCV(estimator = rf_model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [222]:
#rf_Grid.fit(X_train, y_train)

In [223]:
#rf_Grid.best_params_

In [224]:
#print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
#print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

In [225]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid as a dictionary
param_grid = {
    'n_estimators': [300]  # Wrap the value in a list
      # Example of different depths to try
}

# Create the GridSearchCV object with your Random Forest Classifier
rf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)

# Now you can fit the model on the training data


In [226]:
rf_Grid.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [300]})

In [227]:
rf_Grid.best_params_

{'n_estimators': 300}

In [228]:
print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

Train Accuracy - : 1.000
Test Accuracy - : 1.000


In [229]:
"""from sklearn.model_selection import GridSearchCV

# Define the parameter grid as a dictionary
param_grid = {
    'n_estimators': [500]  # Wrap the value in a list
      # Example of different depths to try
}

# Create the GridSearchCV object with your Random Forest Classifier
rf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)

# Now you can fit the model on the training data"""

"from sklearn.model_selection import GridSearchCV\n\n# Define the parameter grid as a dictionary\nparam_grid = {\n    'n_estimators': [500]  # Wrap the value in a list\n      # Example of different depths to try\n}\n\n# Create the GridSearchCV object with your Random Forest Classifier\nrf_Grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)\n\n# Now you can fit the model on the training data"

In [230]:
#rf_Grid.fit(X_train, y_train)

In [231]:
#rf_Grid.best_params_

In [232]:
#print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
#print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

so now lets fine tune the model

In [233]:
# Define Random Forest hyperparameters
n_estimators = 1000  # Number of trees in the forest
max_depth = 10  # Maximum depth of individual trees

# Create Random Forest Classifier object
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

# Train the model on the prepared data
clf.fit(X_train, y_train)

# Make predictions on the testing set (optional, for evaluation)
y_pred = clf.predict(X_test)

# Evaluate the model (example using accuracy)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.9611369677465348


these many iteration are giving a accuracy of approximately of 96%
but it is taking approximately 5 mins 
so lets fine tune the model  for better accuracy in less time 
